# Business Understanding/Бизнес-анализ
## Determine Business Objectives/ Определение бизнес-целей

Не возможно установить. Придумаем задачу.
Допустим по закону на вине должна стоять маркировка качества вина от 0 до 10 и мы не хотим тратить ресурсы на дегустацию. Дать инструмент, который определит качество вина по физ-хим свойствам.

## Assess Situation/ Оценка текущей ситуации
Предположим, пока используется дегустация. Модели нет, но физ-хим данные уже и так собираются.

## Determine Data Mining Goals/ Определение целей аналитики
Обучить, задеплоить и контролировать качество модели, которая будет определять качество вина. 

## ProduktProject Plan/ Подготовка плана проекта

?? видимо, это то, что в файле рядом crispDM_steps.md (все шаги crispDM)

# DataUnderstanding/ Анализ данных

## Collect Initial Data/ Сбор данных 
Скачали данные с kaggle.


## Describe Data/ Описание данных 
Посмотрели описание на kaggle + ниже.

In [2]:
import numpy as np
import pandas as pd
df = pd.read_csv('winequalityN.csv')

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6497 entries, 0 to 6496
Data columns (total 13 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   type                  6497 non-null   object 
 1   fixed acidity         6487 non-null   float64
 2   volatile acidity      6489 non-null   float64
 3   citric acid           6494 non-null   float64
 4   residual sugar        6495 non-null   float64
 5   chlorides             6495 non-null   float64
 6   free sulfur dioxide   6497 non-null   float64
 7   total sulfur dioxide  6497 non-null   float64
 8   density               6497 non-null   float64
 9   pH                    6488 non-null   float64
 10  sulphates             6493 non-null   float64
 11  alcohol               6497 non-null   float64
 12  quality               6497 non-null   int64  
dtypes: float64(11), int64(1), object(1)
memory usage: 660.0+ KB


In [8]:
df['type'].value_counts()

white    4898
red      1599
Name: type, dtype: int64

Действительно тут физ-хим показатели + тип (красное/белое)


## Explore Data/ Изучение данных
## Verify Data Quality/ Проверка качества данных